In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/kaggle/input/car-data-train-test/Datasets/Datasets/Train'
valid_path = '/kaggle/input/car-data-train-test/Datasets/Datasets/Test'

In [ ]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [ ]:
resnet.summary()

In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
folders = glob('/kaggle/input/car-data-train-test/Datasets/Datasets/Train/*')

In [ ]:
len(folders)

In [ ]:
folders

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(folders),activation='softmax')(x)


In [ ]:
# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:


# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   width_shift_range=0.2,
                                    height_shift_range=0.2,
                                   horizontal_flip = True,validation_split=0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:

# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/kaggle/input/car-data-train-test/Datasets/Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:

test_set = test_datagen.flow_from_directory('/kaggle/input/car-data-train-test/Datasets/Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            
                                            
                                            
                                            
                                            class_mode = 'categorical')

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


In [ ]:

# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')




In [ ]:
r.history

In [ ]:

# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_resnet50.h5')


In [ ]:

y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(y_pred)

In [ ]:

y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

In [ ]:

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model = load_model('model_resnet50.h5')

In [ ]:

img=image.load_img('/kaggle/input/car-data-train-test/Datasets/Datasets/Test/lamborghini/1.jpg',target_size=(224,224))

In [ ]:

x=image.img_to_array(img)

In [ ]:
x

In [ ]:
x.shape

In [ ]:
model.predict(x)

In [ ]:
import numpy as np
img = x.reshape(1,224,224,3)

In [ ]:
model.predict(x)

In [ ]:

x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape